In [0]:
#Relevant Links
#1. https://towardsdatascience.com/bert-classifier-just-another-pytorch-model-881b3cf05784
!nvidia-smi


Sun Dec  8 04:18:38 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    31W / 250W |   8287MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
! pip install constant
!pip install bert-pytorch
!pip install pytorch-pretrained-bert pytorch-nlp
!pip install -U -q PyDrive

     |████████████████████████████████| 71kB 3.7MB/s 
  Created wheel for constant: filename=constant-0.0.4-cp36-none-any.whl size=73933 sha256=b6476e8f7dbfffcde5fd3d05534ee572bc0603f9314a2d8881ef3e701ed58f39
  Stored in directory: /root/.cache/pip/wheels/03/ef/ce/18966f544810076cef35c7a0e0eca464b9a3c070164a217f02
Successfully built constant
     |████████████████████████████████| 133kB 4.7MB/s 
     |████████████████████████████████| 92kB 12.8MB/s 
     |████████████████████████████████| 645kB 45.4MB/s 


In [0]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score, precision_score
from sklearn.utils.class_weight import compute_class_weight
from keras.layers import Input, Dense, Dropout
from keras.models import Model
import pandas as pd
import numpy as np
import os, sys
from google.colab import drive

sys.path.append(os.path.join(os.path.dirname(sys.path[0]), 'analysis'))
sys.path.append(os.path.join(os.path.dirname(os.path.dirname(sys.path[0])), 'configs' ))

import constant

Using TensorFlow backend.


In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import warnings
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam, BertModel
from pytorch_pretrained_bert import BertConfig
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


'Tesla P100-PCIE-16GB'

In [0]:
#Data Reading
# Mount drive for data reading
# This will prompt for authorization.
drive.mount('/content/drive')
path = "/content/drive/Shared drives/CIS 519 Project/Code/Dataset/Resampled4/"
torch.cuda.empty_cache()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Read full data path = "/content/drive/Shared drives/CIS 519 Project/Code/Dataset/"
df_train_total = pd.read_csv(path + 'train.csv')
df_val_total = pd.read_csv(path + 'validation.csv')

In [0]:
df_train_total.head(10)

,id,target,comment_text,female,male,homosexual_gay_or_lesbian,christian,jewish,muslim,black,white,psychiatric_or_mental_illness,identity_annotator_count,toxicity_annotator_count,created_date
0,5367475,0.500000,This is an insult to all immigrants who took t...,0.800000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.1,10,10,2017-06-06 17:53:11.976493+00
1,389581,0.166667,"Eric, I was unaware that Obama had a line of p...",0.000000,0.600000,0.000000,0.0,0.0,0.0,0.000000,0.9,0.0,10,6,2016-07-15 23:22:57.603863+00
2,259203,0.600000,"""malign her character because of alleged illeg...",1.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,4,10,2016-03-26 19:14:41.102508+00
3,6274158,0.000000,"Couldn't agree more with your last sentence, E...",0.000000,0.700000,0.900000,0.7,0.0,0.0,0.000000,0.0,0.0,10,4,2017-11-02 15:49:53.222384+00
4,5435081,0.500000,"The liberal, married, gay Black woman who took...",0.833333,0.166667,0.833333,0.0,0.0,0.0,0.833333,0.0,0.0,6,10,2017-06-18 13:26:29.297687+00
5,5693877,0.000000,"Get over it, two men took to the ring and one ...",0.000000,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,4,4,2017-08-01 02:59:30.400500+00
6,5801144,0.600000,"Yes, yes, it's ""The Media"" who forced Trump to...",1.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,1.0,0.0,4,10,2017-08-19 16:02:37.201001+00
7,5078252,0.500000,"""cannon fodder"" - they are ""acceptable losses""...",1.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,4,10,2017-03-31 18:03:36.139268+00
8,5277082,0.000000,"No, not who cares, as in who cares, rather htt...",0.000000,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,4,4,2017-05-19 14:33:37.520110+00
9,489731,0.828571,The hate of white males from the progressive p...,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.000000,1.0,0.0,4,70,2016-09-24 04:44:47.907794+00


In [0]:
df_val_total.head(10)

,id,target,comment_text,female,male,homosexual_gay_or_lesbian,christian,jewish,muslim,black,white,psychiatric_or_mental_illness,identity_annotator_count,toxicity_annotator_count,created_date
0,299192,0.20,I'm pretty sure there's a difference as wide a...,0.800000,0.100000,0.9,0.4,0.0,0.0,0.0,0.0,0.0,10,10,2016-04-30 04:19:50.280309+00
1,6157777,0.70,"5 years as a prisoner of the Taliban? Lol, the...",0.000000,0.700000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,10,2017-10-16 18:44:33.103845+00
2,398687,0.00,"Re: the caption. ""Man wire?"" I think the write...",0.000000,0.833333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,4,2016-07-23 16:19:51.090606+00
3,742714,0.00,Muslims in London want to build Europe's large...,0.000000,0.700000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,10,4,2016-12-22 16:01:05.804576+00
4,927856,0.00,Having been involved with a couple of people w...,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,4,2017-01-31 17:59:56.276643+00
5,6294071,0.00,I think everyone is being way too harsh. Spac...,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,4,2017-11-05 18:06:13.609132+00
6,514611,0.75,"Let's get something straight, skippy, bragging...",1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,80,2016-10-08 05:12:23.719260+00
7,894593,0.50,"Did we really need another instalment in the ""...",0.000000,0.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,10,2017-01-25 11:54:20.923911+00
8,5675570,0.00,These games are a great and worthwhile initiat...,0.800000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,4,2017-07-28 14:44:57.353013+00
9,884943,0.00,You didn't bother to read this did you? The j...,0.833333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,4,2017-01-23 17:53:51.510851+00


In [0]:
# comments = pd.DataFrame(df.comment_text)
# #comments.to_csv(path + "only_comments.csv")
# comments_text = df.comment_text
# df_store = df.copy()
# comments_store = comments.copy()

In [0]:
df_train = df_train_total#.sample(5000)
df_val = df_val_total#.sample(1000)

In [0]:
comments_train = df_train.comment_text
comments_val = df_val.comment_text

In [0]:
#Generate protected attribute - gender
def extract_female_gender(x):
  if np.isnan(x.female) or x.female < 0.5:
      return 0
  else:
    return 1

In [0]:
#Generate unprotected attribute labels
def get_unprotected_class(list_of_protected):
  new = [1 if i == 0 else 0 for i in list_of_protected]
  return new

In [0]:
#Calculate metrics
def get_metrics(labels, preds):
  pred_flat = preds.flatten()
  labels_flat = labels.flatten()

  acc = accuracy_score(labels_flat, pred_flat)
  pre = precision_score(labels_flat, pred_flat)
  rec = recall_score(labels_flat, pred_flat)
  f1 = f1_score(labels_flat, pred_flat, average="weighted")

  return acc, pre, rec, f1

In [0]:
#Generate labels
toxicity_labels_train = list(df_train.target.apply(lambda x: 1 if x >= 0.5 else 0))
identity_labels_train = list(df_train.apply(extract_female_gender, axis = 1))
toxicity_labels_val = list(df_val.target.apply(lambda x: 1 if x >= 0.5 else 0))
identity_labels_val = list(df_val.apply(extract_female_gender, axis = 1))
unprotected_labels_train = get_unprotected_class(identity_labels_train)
unprotected_labels_val = get_unprotected_class(identity_labels_val)

In [0]:
print(len(comments_train), len(toxicity_labels_train))
print(comments_train[:10])
print(toxicity_labels_train[:10])
print(identity_labels_train[:10])

10974 10974
0    This is an insult to all immigrants who took t...
1    Eric, I was unaware that Obama had a line of p...
2    "malign her character because of alleged illeg...
3    Couldn't agree more with your last sentence, E...
4    The liberal, married, gay Black woman who took...
5    Get over it, two men took to the ring and one ...
6    Yes, yes, it's "The Media" who forced Trump to...
7    "cannon fodder" - they are "acceptable losses"...
8    No, not who cares, as in who cares, rather htt...
9    The hate of white males from the progressive p...
Name: comment_text, dtype: object
[1, 0, 1, 0, 1, 0, 1, 1, 0, 1]
[1, 0, 1, 0, 1, 0, 1, 1, 0, 0]


In [0]:
MAX_SEQUENCE_LENGTH = 128
SEED = 519
BATCH_SIZE = 32
BERT_MODEL_PATH = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)

In [0]:
def convert_lines(example, max_seq_length,tokenizer):
    #max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print("Tokens longer than max_length: ", longer)
    return np.array(all_tokens)

In [0]:
#Prepare data
input_train = convert_lines(comments_train.fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)
toxicity_labels_train = torch.tensor(toxicity_labels_train)
female_labels_train = torch.tensor(identity_labels_train)

input_val = convert_lines(comments_val.fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)
toxicity_labels_val = torch.tensor(toxicity_labels_val)
female_labels_val = torch.tensor(identity_labels_val)


  4%|▍         | 88/2000 [00:00<00:02, 876.73it/s]

Tokens longer than max_length:  2690


100%|██████████| 2000/2000 [00:02<00:00, 920.46it/s]

Tokens longer than max_length:  544


In [0]:
print(torch.sum(toxicity_labels_val).data)
print(torch.sum(female_labels_val).data)


tensor(338)
tensor(1144)


In [0]:
#Data Loader
X_train = torch.utils.data.TensorDataset(torch.tensor(input_train, dtype=torch.long), toxicity_labels_train, female_labels_train)
train_loader = torch.utils.data.DataLoader(X_train, batch_size=32, shuffle=True)
#tk0 = tqdm(train_loader)

X_val = torch.utils.data.TensorDataset(torch.tensor(input_val, dtype=torch.long), toxicity_labels_val, female_labels_val)
val_loader = torch.utils.data.DataLoader(X_val, batch_size=32, shuffle=True)
#vk0 = tqdm(val_loader)


In [0]:
#FAIRNESS METRICS FUNCTION

def get_fairness_metrics(actual_labels, y_pred, protected_labels, non_protected_labels, thres):

  def get_toxicity_rates(y_pred, protected_labels, non_protected_labels, thres):
    protected_ops = y_pred[protected_labels == 1]
    protected_prob = sum(protected_ops)/len(protected_ops)

    non_protected_ops = y_pred[non_protected_labels == 1]
    non_protected_prob = sum(non_protected_ops)/len(non_protected_ops)

    return np.round(protected_prob, 2), np.round(non_protected_prob, 2)

  def get_true_positive_rates(actual_labels, y_pred, protected_labels, non_protected_labels, thres):

    protected_ops = y_pred[np.bitwise_and(protected_labels == 1, actual_labels == 1)]
    protected_prob = sum(protected_ops)/len(protected_ops)

    non_protected_ops = y_pred[np.bitwise_and(non_protected_labels == 1, actual_labels == 1)]
    non_protected_prob = sum(non_protected_ops)/len(non_protected_ops)

    return np.round(protected_prob, 2), np.round(non_protected_prob, 2)


  def get_false_positive_rates(actual_labels, y_pred, protected_labels, non_protected_labels, thres):

    protected_ops = y_pred[np.bitwise_and(protected_labels == 1, actual_labels ==0)]
    protected_prob = sum(protected_ops)/len(protected_ops)

    non_protected_ops = y_pred[np.bitwise_and(non_protected_labels == 1, actual_labels == 0)]
    non_protected_prob = sum(non_protected_ops)/len(non_protected_ops)

    return np.round(protected_prob, 2), np.round(non_protected_prob, 2)

  def demographic_parity(y_pred, protected_labels, non_protected_labels, thres):

    protected_ops = y_pred[protected_labels == 1]
    protected_prob = sum(protected_ops)/len(protected_ops)

    non_protected_ops = y_pred[non_protected_labels == 1]
    non_protected_prob = sum(non_protected_ops)/len(non_protected_ops)

    return abs(protected_prob - non_protected_prob) #later take absolute diff - but we want to show females predicted more toxic than male

  # | P_female(C = 1| Y = 1) - P_male(C = 1 | Y = 1) | < thres
  def true_positive_parity(actual_labels, y_pred, protected_labels, non_protected_labels, thres):

    protected_ops = y_pred[np.bitwise_and(protected_labels == 1, actual_labels == 1)]
    protected_prob = sum(protected_ops)/len(protected_ops)

    non_protected_ops = y_pred[np.bitwise_and(non_protected_labels == 1, actual_labels == 1)]
    non_protected_prob = sum(non_protected_ops)/len(non_protected_ops)

    return abs(protected_prob - non_protected_prob) #later take absolute diff - but we want to show females predicted more toxic than male

  # | P_female(C = 1| Y = 0) - P_male(C = 1 | Y = 0) | < thres
  def false_positive_parity(actual_labels, y_pred, protected_labels, non_protected_labels, thres):

    protected_ops = y_pred[np.bitwise_and(protected_labels == 1, actual_labels ==0)]
    protected_prob = sum(protected_ops)/len(protected_ops)

    non_protected_ops = y_pred[np.bitwise_and(non_protected_labels == 1, actual_labels == 0)]
    non_protected_prob = sum(non_protected_ops)/len(non_protected_ops)

    return abs(protected_prob - non_protected_prob) #later take absolute diff - but we want to show females predicted more toxic than male


  # Satisfy both true positive parity and false positive parity
  def equalized_odds(actual_labels, y_pred, protected_labels, non_protected_labels, thres):
    return true_positive_parity(actual_labels, y_pred, protected_labels, non_protected_labels, thres) + false_positive_parity(actual_labels, y_pred, protected_labels, non_protected_labels, thres)

  female_tox_rate, nf_tox_rate = get_toxicity_rates(y_pred, protected_labels, non_protected_labels, thres)
  female_tp_rate, nf_tp_rate = get_true_positive_rates(actual_labels, y_pred, protected_labels, non_protected_labels, thres)
  female_fp_rate, nf_fp_rate = get_false_positive_rates(actual_labels, y_pred, protected_labels, non_protected_labels, thres)
  demo_parity = demographic_parity(y_pred, protected_labels, non_protected_labels, thres)
  tp_parity = true_positive_parity(actual_labels, y_pred, protected_labels, non_protected_labels, thres)
  fp_parity = false_positive_parity(actual_labels, y_pred, protected_labels, non_protected_labels, thres)
  equ_odds = equalized_odds(actual_labels, y_pred, protected_labels, non_protected_labels, thres)

  return female_tox_rate, nf_tox_rate, female_tp_rate, nf_tp_rate, female_fp_rate, nf_fp_rate, demo_parity, tp_parity, fp_parity, equ_odds



In [0]:
"""
UnfairClassifier - BERT  - PyTorch Implementation
"""
config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
        num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072, 
        hidden_dropout_prob=0.1)

class Regular_Classifier(nn.Module):
    def __init__(self, toxicity_labels = 2):
        super(Regular_Classifier, self).__init__()
        
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.c1 = nn.Linear(config.hidden_size, 324)
        #self.c2 = nn.Linear(config.intermediate_size, 324)
        self.c3 = nn.Linear(324, 2)

        nn.init.xavier_normal_(self.c1.weight)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):

        
        #BERT
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        pooled_output = self.dropout(pooled_output)

        # Classifier
        classifier = F.relu(self.c1(pooled_output))
        #classifier = F.relu(self.c2(classifier))
        classifier_output = self.c3(classifier)


        return classifier_output

net = Regular_Classifier(toxicity_labels = 2) # instantiate the nn

In [0]:
def conduct_validation(net, data_loader, adv = False):

    eval_loss, eval_accuracy, eval_precision, eval_recall, eval_f1 = 0, 0, 0, 0, 0
    nb_eval_steps = 0
    
    predictions_net = np.empty((0,))
    truths = np.empty((0,))
    identities = np.empty((0,))
    correct_net = 0
    total = 0

    net.eval()
    with torch.no_grad(): # IMPORTANT: we don't want to do back prop during validation/testing!
      for index, data in enumerate(data_loader):

        text, toxic_truth, female_truth = data

        text = text.to(device)
        toxic_truth = toxic_truth.to(device)
        female_truth = female_truth.to(device)

        if adv:
          net_outputs, net_prev_outputs = net(text)
        else:
          net_outputs = net(text)
        _, net_predicted = torch.max(net_outputs.data, 1)

        batch_size = toxic_truth.size(0)
        total += batch_size
        correct_net_batch = (net_predicted == toxic_truth).sum().item()
        correct_net += correct_net_batch

        
        predictions_net = np.concatenate((predictions_net, net_predicted.cpu().numpy()))
        truths = np.concatenate((truths, toxic_truth.cpu().numpy()))
        identities = np.concatenate((identities, female_truth.cpu().numpy()))

        pred = net_predicted.detach().cpu().numpy()
        label_ids = toxic_truth.to('cpu').numpy()

        tmp_eval_accuracy, tmp_eval_precision, temp_eval_recall, tmp_eval_f1 = get_metrics(label_ids, pred)

        eval_accuracy += tmp_eval_accuracy
        eval_precision += tmp_eval_precision
        eval_recall += temp_eval_recall
        eval_f1 += tmp_eval_f1
        nb_eval_steps += 1

    f1_score = eval_f1/nb_eval_steps
    prec_score = eval_precision/nb_eval_steps
    recall_score = eval_recall/nb_eval_steps
    acc_score = eval_accuracy/nb_eval_steps

    print("F1 Score: ", f1_score)
    print("Precision Score: ", prec_score)
    print("Recall Score: ", recall_score)
    print("Acc Score: ", acc_score, "\n\n")

    net.train()
    
    return (predictions_net, truths, identities, acc_score)

In [0]:
### REGULAR CLASSIFIER

lrlast = .001
lrmain = .00001
optimizer_net = optim.Adam(
    [
        {"params":net.bert.parameters(),"lr": lrmain},
        {"params":net.c1.parameters(), "lr": lrlast},
        #{"params":clf.c2.parameters(), "lr": lrlast},
    {"params":net.c3.parameters(), "lr": lrlast}    
  ])
  

loss_criterion = torch.nn.CrossEntropyLoss()

# Training Process

net.to(device)


train_accs = []
valid_accs = []

n_epochs = 3

for epoch in range(n_epochs):  # loop over the dataset multiple times


    running_net_loss = 0.0
    
    print('Epoch:' + str(epoch))

    for i, data in enumerate(train_loader): # starting from the 0th batch
        # get the inputs and labels
        
        inputs, toxicity_true, female_true = data
        inputs = inputs.to(device)
        toxicity_true = toxicity_true
        female_true = female_true
        #toxicity_true = torch.sparse.torch.eye(2).index_select(dim=0, index=toxicity_true) 
        #female_true = torch.sparse.torch.eye(2).index_select(dim=0, index=female_true) 

        #print(toxicity_true.dtype)

        toxicity_true = toxicity_true.to(device)
        female_true = female_true.to(device)


        optimizer_net.zero_grad()

        net_output = net(inputs)
        net_loss = loss_criterion(net_output, toxicity_true) # compute loss
        net_loss.backward()
        optimizer_net.step()

        running_net_loss += net_loss.item()

        if i % 32 == 31:   
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_net_loss / 32))
            
            running_total_loss = 0.0
            running_net_loss = 0.0


    print('Training metrics:')
    y_pred, actual_labels, protected_labels, acc_score = conduct_validation(net, train_loader, adv = None)
    train_accs.append(acc_score)

    print('Validation metrics:')
    y_pred, actual_labels, protected_labels, acc_score = conduct_validation(net, val_loader, adv = None)
    valid_accs.append(acc_score)

    if epoch == n_epochs-1:

      non_protected_labels = np.asarray(get_unprotected_class(protected_labels))
      thres = 0.5
      female_tox_rate, nf_tox_rate, female_tp_rate, nf_tp_rate, female_fp_rate, nf_fp_rate, demo_parity, tp_parity, fp_parity, equ_odds =\
       get_fairness_metrics(actual_labels, y_pred, protected_labels, non_protected_labels, thres)

      print("Toxicity Prediction Rates: ", "Female -", female_tox_rate, "Non-Female - ", nf_tox_rate)
      print("True Positive Prediction Rates: ", "Female -", female_tp_rate, "Non-Female - ", nf_tp_rate)
      print("False Positive Prediction Rates: ", "Female -", female_fp_rate, "Non-Female - ", nf_fp_rate)
      print("Demographic Parity: ", demo_parity)
      print("True Positive Parity: ", tp_parity)
      print("False Positive Parity: ", fp_parity)
      print("Equalized Odds: ", equ_odds)


Epoch:0
[1,    32] loss: 0.742
[1,    64] loss: 0.698
[1,    96] loss: 0.684
[1,   128] loss: 0.683
[1,   160] loss: 0.602
[1,   192] loss: 0.506
[1,   224] loss: 0.449
[1,   256] loss: 0.466
[1,   288] loss: 0.477
[1,   320] loss: 0.412
Training metrics:
F1 Score:  0.8522093573614691
Precision Score:  0.8379454678958149
Recall Score:  0.8718289219548374
Acc Score:  0.8522959183673469 


Validation metrics:
F1 Score:  0.7564512664169679
Precision Score:  0.3581232609593954
Recall Score:  0.828557685700543
Acc Score:  0.7217261904761905 


Epoch:1
[2,    32] loss: 0.386
[2,    64] loss: 0.337
[2,    96] loss: 0.335
[2,   128] loss: 0.357
[2,   160] loss: 0.342
[2,   192] loss: 0.331
[2,   224] loss: 0.382
[2,   256] loss: 0.365
[2,   288] loss: 0.330
[2,   320] loss: 0.334
Training metrics:
F1 Score:  0.8744975058548475
Precision Score:  0.9484038529645497
Recall Score:  0.7909610176886892
Acc Score:  0.875522351797862 


Validation metrics:


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


F1 Score:  0.8512964875082815
Precision Score:  0.5116991914610962
Recall Score:  0.6294904830619117
Acc Score:  0.84375 


Epoch:2
[3,    32] loss: 0.283
[3,    64] loss: 0.296
[3,    96] loss: 0.258
[3,   128] loss: 0.263
[3,   160] loss: 0.272
[3,   192] loss: 0.248
[3,   224] loss: 0.265
[3,   256] loss: 0.286
[3,   288] loss: 0.266
[3,   320] loss: 0.262
Training metrics:
F1 Score:  0.9450966027614701
Precision Score:  0.9545056111760863
Recall Score:  0.9330901734420185
Acc Score:  0.9450619533527697 


Validation metrics:
F1 Score:  0.8263218753192836
Precision Score:  0.46213002165383105
Recall Score:  0.7547871000251952
Acc Score:  0.8060515873015873 


Toxicity Prediction Rates:  Female - 0.35 Non-Female -  0.2
True Positive Prediction Rates:  Female - 0.84 Non-Female -  0.65
False Positive Prediction Rates:  Female - 0.25 Non-Female -  0.1
Demographic Parity:  0.1490180380367296
True Positive Parity:  0.18424306659600775
False Positive Parity:  0.15353238096226957
Equalized 

In [0]:
torch.save(net.state_dict(), path + "Final_Regular_Classifier")

In [0]:
del net

torch.cuda.empty_cache()